In [19]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy

In [20]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [21]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k
    def __getitem__(self, key):
        return self.rows[key]
    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

In [22]:
import numpy as np

def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)

def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


def fitness_function(state):
        tmp = deepcopy(state)
        return nim_sum(tmp)

def GaussianMutation(state, sigma, mutation_probability): #randomic mutation of elements
    mutated_ply = []
    for r, c in enumerate(state.rows):
        for o in range(1, c + 1):
            if state.rows[r] >= o and np.random.rand() < mutation_probability:
                mutated_value = int(o + o * np.random.normal(0, sigma))
                mutated_ply.append((r, mutated_value))
            else:
                mutated_ply.append((r, o))
    return mutated_ply

def Recombination(state):

       p1 = [(r,o) for r, c in enumerate(state.rows) for o in range(1, c + 1)  if c > 0 if c>=o]
       p2 = [(r,o) for r, c in enumerate(state.rows)if c > 0 for o in range(1, c + 1)  if c>=o]

       if p2!=[]:
        indice = random.randint(0, len(p1) - 1)
        recombinated_ply = list(p1[:indice]) + list(p2[indice:])
        return recombinated_ply
       return p1

def apply_move(current_state, move):
    row, num_objects = move
    # Verify if the move is good
    if  1 <= num_objects <= current_state[row]:
        # Apply the move
        current_state._rows[row] -= num_objects
        return current_state
    else:
        # move is not good
        return current_state

def fitness_function_new(current_state,strategy):
    # calculate the nim_sum and obtain new move
    new_state = apply_move(current_state, strategy)
    
    if not isinstance(new_state,(int,float)):
      nim_sum_value = nim_sum(new_state)
    else:
        nim_sum_value=0

     
    inverted_fitness = nim_sum_value
    return inverted_fitness

def tournament_selection(population,state):
    
    tournament_winner = None
    best_fitness =0
    stato_farlocco=deepcopy(state)
    tournament_participants = random.sample(population,len(population)) 
      
    # Valuta le tuple di mosse e trova il vincitore del torneo tremaghi
    for (row, num_objects) in tournament_participants:
        fitness_value = fitness_function_new(stato_farlocco,(row, num_objects))
        if fitness_value > best_fitness:
            best_fitness = fitness_value
            tournament_winner = (row, num_objects)
    return (tournament_winner,best_fitness)

def es(state, population_size, mutation_rate, sigma, lambda_value):


    best_strategy = None
    best_score=float('inf')
    weights = {'best_score': float('inf'), 'current_weight': 1.0}

    for _ in range(population_size):

        if random.random() < mutation_rate:
            population = GaussianMutation(state, sigma,mutation_rate)
        else:
            population = Recombination(state)
        
        for strategy in population:
              
            if isinstance(strategy, tuple) and len(strategy) == 2:
                score=fitness_function(state)
            if score<best_score and strategy[1] > 0 and strategy[1] <= state.rows[strategy[0]]:
                best_score=score
                best_strategy = strategy
        # Make λ new individuals casually and update it as a population part
        for _ in range(lambda_value):
            row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
            num_objects = random.randint(1, state.rows[row])
            new_individual = (row, num_objects)
            population.append(new_individual)

    if best_strategy is None:
        row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
        num_objects = random.randint(1, state.rows[row])
        return (row, num_objects)

    return best_strategy

def es_with_tournament(state, population_size, mutation_rate, sigma, lambda_value):

    best_strategy = None
    lista=[]

    for _ in range(population_size):
        if random.random() < mutation_rate:
            population = GaussianMutation(state, sigma,mutation_rate)
        else:
            population = Recombination(state)

        vincitore,valore_fitness=tournament_selection(population,state)
        
        lista.append((vincitore,valore_fitness))

        # Make λ new individuals casually and update it as a population part
        for _ in range(lambda_value):
            row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
            num_objects = random.randint(1, state.rows[row])
            new_individual = (row, num_objects)
            population.append(new_individual)
   

    
    lista_ordinata_decrescente = sorted(lista, key=lambda x: x[1], reverse=True)
    buona_strategia=lista_ordinata_decrescente[0][0]
    # Trova la tupla con il secondo elemento massimo
   
    if buona_strategia[1]<=state.rows[buona_strategia[0]] and buona_strategia is not None:
        best_strategy=buona_strategia
             
    if best_strategy is None:
        row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
        num_objects = random.randint(1, state.rows[row])
        return (row, num_objects)

    return best_strategy



In [23]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
   
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

def salvatore(state:Nim) -> Nimply:
    migliore_strategia=es_with_tournament(state,100,0.3,0.5,5)
    row=migliore_strategia[0]
    n_objects=migliore_strategia[1]
    return Nimply(row,n_objects)

In [24]:
logging.getLogger().setLevel(logging.INFO)
nim = Nim(5)
stato_iniziale=deepcopy(nim)
strategy = (optimal,salvatore)
logging.info(f"init : {nim}")
p = 0
while stato_iniziale:
    if p==0:
        
        ply=strategy[0](stato_iniziale)
          
    else:
         ply=strategy[1](stato_iniziale)

    logging.info(f"ply: player {p} plays {ply}")
    stato_iniziale.nimming(ply) 
    logging.info(f"status: {stato_iniziale}")
    player = 1 - p
    if player==0:
        p=0
    else:
        p=1
logging.info(f"status: Player {p} won!")

INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=5)
INFO:root:status: <1 3 5 7 4>


INFO:root:ply: player 1 plays Nimply(row=3, num_objects=3)
INFO:root:status: <1 3 5 4 4>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=1)
INFO:root:status: <1 3 4 4 4>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=3)
INFO:root:status: <1 3 1 4 4>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=1)
INFO:root:status: <1 3 1 3 4>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=2)
INFO:root:status: <1 1 1 3 4>
INFO:root:ply: player 0 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 1 1 3 4>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=1)
INFO:root:status: <0 1 1 2 4>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=3)
INFO:root:status: <0 1 1 2 1>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=0)
INFO:root:status: <0 1 1 2 1>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=1)
INFO:root:status: <0 0 1 2 1>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=1)
INFO:root:status: <0 0 1 2 0>
INFO:root:ply: player